# What is JugleChart?
JugleChart is Python/jupyter api for creating a Google Chart using generated javascript and html that calls the Google Charts library.  JugleChart generates the javascript and html for you.  You don't have to worry about loading the chart library, creating valid javascript or html, or creating unique id's for your chart containers.

*** Note: JugleChart is a thin wrapper for Google Charts.  You must consult Google Charts documenation for all types and options to pass to JugleChart.  It does not replace the Google Charts documentation.***

JugleChart is designed to be simple, so it has some limitations over coding your own Google Chart.
* JugleChart accepts only one set of data.
* JugleChart can create only one chart.
* JugleChart does not support javascript event handling.

JugleChart will allow you to do with a chart whatever the most generic Google Chart javascript structure permits.  It will plug your selections into a basic javascript template.  In other words, any charts and options that do not require specially coded javascript should work in JugleChart.  What JugleChart will let you do:
* Use most (if not all) Google Chart types.
* Use most (if not all) Google Chart configuration options.
* Apply one or more of Google Chart display formatters.
* Create one or more dashboard controls that can bind to the data or to other controls.


# Dependencies
The following packages must be installed.
* pandas
* ipython
* gviz_api
* JugleChart

# The import statements

In [1]:
# There is one required import.
from juglechart_api import JugleChart

# Optional import, if you're using pandas to format your data.
import pandas as pd

# Optional import, if you're going to call the display methods yourself, rather than using .show()
from IPython.display import display, HTML 


# Quick Start

In [2]:
# create a chart object from a data frame.
data = pd.DataFrame({'cities': ['Chicago', 'Atlanta'], 'widgets': [35, 23]})
chart = JugleChart(data)

# call .show()
chart.add_chart_options(title="asdfadf", is3d=True)
chart.show("ColumnChart") 
# default is line chart.

# 1. Adding data
Data must be passed to the constructor when you create a GChart.  Data can be passed to the constructor in three ways.
* as a 2d list
* as a pandas DataFrame
* as two or more pandas Series.

## Creating data as 2-d list
The data is passed as a list of lists.  The first inner list is the headers.  Each list following contains the data.  The first item in a data list is usually the category; each following item is a plotted value.

In [3]:
data = []

# The first list is the column headings
data.append(['Kid', 'cupcakes', 'candy bars'])

# Each list after that is a row of data.
data.append(['Bobby', 3, 10])
data.append(['Cindy', 2, 7])
data.append(['Marsha', 4, 5])
data.append(['Greg', 5, 8])

chart = JugleChart(data)

## Creating data as a DataFrame
Basically, the rows of a DataFrame are rows of data.  For many chart types (e.g, BarChart, ColumnChart), the first column of the DataFrame contains the categories for the chart, and each column after that has plotted values.  For other charts (like ScatterChart), the first column contains a coordinate value on one axis, and the subsequent columns contain coordinate values on the other axis.

** The index is ignored by JugleChart.  If you would like the index to be used as the categories, you must reset the index on the DataFrame.**

In [4]:
kids = ['Greg', 'Marsha', 'Jan', 'Peter', 'Cindy', 'Bobby']
sodas = [3, 5, 4, 7, 3, 2]
burgers = [4, 6, 5, 2, 7, 8]

data = pd.DataFrame({'kids': kids, 'sodas': sodas, 'burgers': burgers}, columns=['kids', 'sodas', 'burgers'])

chart = JugleChart(data)

## Passing data from two or more Series
Selected columns of a DataFrame can be passed as Series.  The first Series is the categories or the x-values, say.  The following Series hold the y-values.

In [5]:
# pass DataFrame columns
chart = JugleChart(data.kids, data.burgers)


## Data types supported
In the javascript, data is contained in a DataTable object.  DataTable accepts the following the data types:  boolean, number, string, date, datetime, and timeofday.  When passing data to JugleChart in a 2d array, the array is turned into a DataTable with the javascript method .arrayToDataTable().  The datatypes can be specified in the 2d array, as shown in https://developers.google.com/chart/interactive/docs/datatables_dataviews#arraytodatatable.  Or, the data types can be unspecified, leaving the DataTable() constructor to infer the datatypes.

When data is passed to JugleChart in a DataFrame or Series, the javascript datatypes will be explicitly set according to the dtypes on the pandas object.  The mapping of dtypes to DataTable() types is straightforward, with two caveats.
1.  timeofday is not currently supported.
2.  All datetimes in pandas will be converted to dates (truncating h:m:s where necessary), unless columns are specified to be dateimes with a special keyword:  **datetime_cols=[2, 3]**

## None/NaN/NaT in a DataFrame
If there is a None, NaN, or NaT in a DataFrame or Series passed to Ghart, an exception is raised, warning about the missing values.  If you would like these values to figure into the chart, pass the keyword argument

**allow_nulls=True**

in the JugleChart constructor.  Any of the values -- None, NaN, or NaT -- that appear are converted to **null** in the javascript.

# 2. Selecting a chart type
The chart has a default type of ColumnChart.  A chart type can be assigned at instantiation with the keyward argument **chart_type**.

***An extensive list of chart types is shown in the Google Charts documentation.***

In [6]:
chart = JugleChart(data, chart_type = 'BarChart')

Alternatively, a chart's type can be changed at any time by simply setting the **chart_type** property

In [7]:
chart.chart_type = 'LineChart'

# 3. Generating a chart
## .render()
Calling **.render()** on the chart object will generate javascript and html that can be displayed with ipython functions.

A chart can be displayed (and modified) multiple times in a notebook, as each chart container div will have it's own unique id.

In [8]:
display(HTML(chart.render()))

A chart type can be passed to the render() function to display a chart type on the fly without changing
the underlying chart type.

In [9]:
display(HTML(chart.render('BarChart')))

## .show()

For easier display, a chart has a convenience function -- **.show()** -- that wraps *.render()* and ipython's *display(HTML())*

In [10]:
chart.show()
chart.show("ColumnChart")

# 4. Chart Options

The chart can be given any of the configuration options listed in the Google Charts documentation, such as chart title or size, axis attributes, or plot attributes.  

*** The list of configuration options can be found in the Google Charts documentation for any chart type, e.g. BarChart:*** https://developers.google.com/chart/interactive/docs/gallery/barchart#configuration-options

Chart options are passed in the **add_chart_options()** method.  There are two ways to pass options.  One or more options can be passed in dictionary.  Or, they can be passed as keyword arguments.

Some configuration options are shown in the documentation as a chain of nested javascript objects -- in dot notation, and in nested braces. For example, to set the color of the horizontal axis to blue, you would use the setting

**{hAxis: {titleTextStyle: {color: "blue"} } }**

There are several ways to set a chained property with GChart.
* As a set of nested dictionaries, paralleling the javascript object notation.
* As a single dictionary, where the keyword has dot notation.
* As a keyword argument, where the keyword uses '\_' as the chain links.

To set an option value to javascript true or false, pass the Python objects True or False

To set an option value to javascript null, pass the Python object None.

## Passing options as a dictionary

In [11]:
chart.add_chart_options({'width': '600px'})

# Adding options is cumulative.
# You can pass one or more options in one dictionary.
chart.add_chart_options({'height':  '400px', 'width': '600px'})

# Add a chained option as a fully formed dictionary...
chart.add_chart_options({'hAxis': {'title': 'Kids'}})
chart.add_chart_options({'hAxis': {'titleTextStyle': {'color': 'green'}}})

# Or, add chained option with dot notation
options = {}
options['vAxis.title'] = 'Treats'
options['vAxis.titleTextStyle.color'] = 'red'
chart.add_chart_options(options)

chart.show()

## Passing options as keyword args

In [12]:
chart.add_chart_options(width = '600px', height = 400)

# Use underscore notation to add a chained option
chart.add_chart_options(hAxis_titleTextStyle_color = 'blue')

## Reseting all options
To reset all chart options, set

**chart.chart_options = None**

This is useful in development when you are testing styles by adding and removing them.  Unless you restart the kernel, an option remains attached to the chart unless an option is set to None, or all chart options are reset.

# 5. Chart div container styles
CSS Styles can be applied to the div that contains the chart. Pass these in a dictionary or as keyword arguments.

In [13]:
# Add styles with dictionary
chart.add_div_styles({'height': '400px'})

# Or, add styles with keyword arg
chart.add_div_styles(width = '500px')